# 0 - Manipulação e Peparação dos dados

## Importação dos módulos

In [1]:
import cv2 
import os
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## Configurações e Constantes

## Declaração de funções

In [ ]:
#Recebe uma lista de nomes de colunas e cria um dataframe vazio
def createEmptyDf(column_names):
    return pd.DataFrame(columns=column_names)

# função que retorna se o arquivo é JPG
def isJPG(file_name):
    if file_name.endswith('jpg') or file_name.endswith('JPG'):
        return True
    else: 
        return False
        
#Recebe um diretorio e retorna uma lista de imagens JPG e uma lista com arquivos XML
def loadImageAndAnnotation(dir_path):
    
    # Carregar as Arquivos (JPGs e XML)
    file_names_list_full = os.listdir(dir_path)

    # Separar JPG de XML
    images_names_list = [nome for nome in file_names_list_full if isJPG(nome)]
    annotation_names_list = [nome for nome in file_names_list_full if not isJPG(nome)]
    
    return images_names_list, annotation_names_list

#Esta função converte as anotações de XML para CSV
def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

## Convertendo as anotações .xml em .csv

In [ ]:
import numpy as np
import json
import pandas as pd
import os
import glob
import xml.etree.ElementTree as ET

#### Configurações e constantes

In [ ]:
ORIG_DATASET_PATH = "C:\\NEW_TCC\\dataset\\original"
OUTPUT_ORIG_DATASET_PATH = ORIG_DATASET_PATH

#### RunJob

In [ ]:
def xmlToCsvJob(input_path, output_path):
    df = xml_to_csv(input_path)

    df.to_csv(os.path.join(output_path, "orig_annotation.csv"), encoding='utf-8', index=False)

    print("CSV criado com Sucesso!")

xmlToCsvJob(ORIG_DATASET_PATH, OUTPUT_ORIG_DATASET_PATH)

# 1 - Separação do conjuto de dados de Validação

#### Importação dos módulos

In [ ]:
import random
import pandas as pd

#### Definição de funções

#### Configuração e Constantes

In [ ]:
ORIG_DATASET_PATH = "C:\\NEW_TCC\\dataset\\original"
SELECT_DATASET_PATH = "C:\\NEW_TCC\\dataset\\selecionados"
VAL_DATASET_PATH = "C:\\NEW_TCC\\dataset\\validacao"


ORIG_ANNOTATION_PATH = os.path.join(ORIG_DATASET_PATH, "orig_annotation.csv")
SEL_ANNOTATION_PATH = os.path.join(SELECT_DATASET_PATH, "sel_annotation.csv")
VAL_ANNOTATION_PATH = os.path.join(VAL_DATASET_PATH, "val_annotation.csv")



FRACAO_VALIDACAO = 15

In [ ]:
def main():
    
    # Cria um Dataframe vazio para armazenar as annotations das Imagens de treino
    row = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax']
    final_annotation_train_df = pd.DataFrame(row, columns=column_name)
    
    # Cria um Dataframe vazio para armazenar as annotations das Imagens de teste
    row = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax']
    final_annotation_test_df = pd.DataFrame(row, columns=column_name)
    
    
    
    image_names_list = createListNameImageFiles(ORIG_DATASET_PATH)
    
    Qt_image = len(image_names_list)
    
    n_test = int(Qt_image*FRACAO_VALIDACAO/100)
    
    test_list = random.sample(image_names_list, n_test)
    
    train_list = list(set(image_names_list) - set(test_list))
    
    for img in test_list:
        !copy {os.path.join(img)} {os.path.join(VAL_DATASET_PATH)}   
        
    
    # carrega o csv com as annotations
    annotation_df = pd.read_csv(ORIG_ANNOTATION_PATH)
    
    # criar Test_Annotation_DF
    for path in test_list:
        image_name = createNameImage(path)
        imageAnnotationDf = annotation_df[annotation_df["filename"]==image_name+".JPG"]
        
        final_annotation_test_df = pd.concat([final_annotation_test_df, imageAnnotationDf])
        
    # Salva o csv com as Annotations das imagens de Teste
    final_annotation_test_df = final_annotation_test_df.reset_index(drop=True)
    final_annotation_test_df.to_csv(VAL_ANNOTATION_PATH, encoding='utf-8', index=False)
    
    for img in train_list:
        !copy {os.path.join(img)} {os.path.join(SELECT_DATASET_PATH)} 
    
    # criar Train_Annotation_DF
    for path in train_list:
        image_name = createNameImage(path)
        imageAnnotationDf = annotation_df[annotation_df["filename"]==image_name+".JPG"]
        
        final_annotation_train_df = pd.concat([final_annotation_train_df, imageAnnotationDf])
        
    # Salva o csv com as Annotations das imagens de Treino
    final_annotation_train_df = final_annotation_train_df.reset_index(drop=True)
    final_annotation_train_df.to_csv(SEL_ANNOTATION_PATH, encoding='utf-8', index=False)
        
    

main()    

#### Renomeando os arquivos de Validação

# 2 - Janelamento

Este notebook tem como objetivo:
- Separar a imagem original em multiplas janelas menores
- Criar uma nova annotation para a janela
- Separar as janelas com o objeto de iteresse (positivas) e sem o objetos de interesse (negativas)

## Importação dos módulos

In [2]:
import random
import numpy as np
import json
import pandas as pd
import os
import cv2 
import glob
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
%matplotlib inline

## definição de funções

In [3]:
# Esta função recebe um caminho e cria uma pasta nesse diretorio caso não exista
def createDir(new_path):
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        print("diretório " +  new_path + " criado com sucesso")
        
#Cria uma Lista com o Nome de todas as imagens no path de entrada 
def createListNameImageFiles(input_path):

    from pathlib import Path
    images = Path(input_path).glob("*.jpg")
    image_names_list = [str(p) for p in images]

    return image_names_list

# Esta função retonar apenas o nome da imagem a partir de um path
def createNameImage(paths_image):
    name_format = paths_image.split("\\")[-1]
    image_name=name_format.split(".")[0] 
    
    return image_name


def DefineNewBoundingBoxAnnotation(X1_roi, X2_roi, Y1_roi, Y2_roi, x1, x2, y1, y2, x1_flag, x2_flag, y1_flag, y2_flag):
    
    if x1_flag:
        x1_new = x1
    else:
        x1_new = X1_roi if x1 < X1_roi else X2_roi
        
    if x2_flag:
        x2_new = x2
    else:
        x2_new = X2_roi if x2 > X2_roi else X1_roi 
        
    if y1_flag:
        y1_new = y1
    else:
        y1_new = Y1_roi if y1 < Y1_roi else Y2_roi
        
    if y2_flag:
        y2_new = y2
    else:
        y2_new = Y2_roi if y2 > Y2_roi else Y1_roi     
        
    x1_new = x1_new - X1_roi
    y1_new = y1_new - Y1_roi
    x2_new = x2_new - X1_roi
    y2_new = y2_new - Y1_roi
    
    return x1_new, y1_new, x2_new, y2_new 

def create_empty_df():
    # Cria um Dataframe vazio para armazenar as annotations das janelas para todas as imagens
    final_annotation_windows_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax', 'pc_match','pc_match_flag']
    final_annotation_windows_full_df = pd.DataFrame(final_annotation_windows_list, columns=column_name)
    
    return final_annotation_windows_full_df

# avalia a porcentagem da sobreposição 
def overlappingAreaFilter(
    x1_new,
    y1_new,
    x2_new,
    y2_new,
    x1,
    x2,
    y1,
    y2):
    
    Area_Total = (x2-x1)*(y2-y1)
    Area_Interseccao = (x2_new-x1_new)*(y2_new-y1_new)
    
    porcentagem_sobreposta = (Area_Interseccao/Area_Total)*100
    
    pc_match_flag: str = ""
    
    
    # match_100
    if porcentagem_sobreposta >= 100:
        pc_match = porcentagem_sobreposta
        pc_match_flag = "MATCH_100"
    
    # match_95_mais
    if porcentagem_sobreposta >= 95 and porcentagem_sobreposta < 100:
        pc_match = porcentagem_sobreposta
        pc_match_flag = "MATCH_95_MAIS"
    
    # match_80_mais
    if porcentagem_sobreposta >= 80 and porcentagem_sobreposta < 95:
        pc_match = porcentagem_sobreposta
        pc_match_flag = "MATCH_80_MAIS"
    
    # match_80_menos
    if porcentagem_sobreposta < 80:
        pc_match = porcentagem_sobreposta
        pc_match_flag = "MATCH_80_MENOS"
        
    
    return pc_match, pc_match_flag

def exportWindows(windows_img, DIRETORY_OUTPUT_PATH, windows_name):
    OUTPUT_PATH = os.path.join(DIRETORY_OUTPUT_PATH, windows_name + ".JPG")
    cv2.imwrite(OUTPUT_PATH, cv2.cvtColor(windows_img, cv2.COLOR_RGB2BGR))
    
def exportWindowsNoCow(windows_img, DIRETORY_OUTPUT_NOCOW_PATH, i_NoCow):
    OUTPUT_PATH = os.path.join(DIRETORY_OUTPUT_NOCOW_PATH, "NoCow_" + str(i_NoCow) + ".JPG")
    cv2.imwrite(OUTPUT_PATH, cv2.cvtColor(windows_img, cv2.COLOR_RGB2BGR))
    
def BoundingBoxDetectionFilter(
    image_name,
    windows_name,
    annotation_df, 
    X1_roi,
    Y1_roi,
    X2_roi,
    Y2_roi,
    windows_width,
    windows_height):
    
    
    imageAnnotationDf = annotation_df[annotation_df["filename"]==image_name+".JPG"]
    intersection_flag = False    
   
    windows_annotation_list = []
   
    for obj in range(len(imageAnnotationDf)):
        x1 = imageAnnotationDf.iloc[obj][4]
        y1 = imageAnnotationDf.iloc[obj][5]
        x2 = imageAnnotationDf.iloc[obj][6]
        y2 = imageAnnotationDf.iloc[obj][7]
        
        
        x1_flag = True if x1 >= X1_roi and x1 <= X2_roi else False
        x2_flag = True if x2 >= X1_roi and x2 <= X2_roi else False
        y1_flag = True if y1 >= Y1_roi and y1 <= Y2_roi else False
        y2_flag = True if y2 >= Y1_roi and y2 <= Y2_roi else False

        if ((x1_flag or x2_flag) and (y1_flag or y2_flag)):
            intersection_flag = True
            x1_new, y1_new, x2_new, y2_new = DefineNewBoundingBoxAnnotation(
                X1_roi,
                X2_roi,
                Y1_roi,
                Y2_roi,
                x1,
                x2,
                y1,
                y2,
                x1_flag,
                x2_flag,
                y1_flag,
                y2_flag)
            
            pc_match, pc_match_flag = overlappingAreaFilter(
                x1_new,
                y1_new,
                x2_new,
                y2_new,
                x1,
                x2,
                y1,
                y2)
            
            
            value = (windows_name, windows_width, windows_height, imageAnnotationDf.iloc[obj][3], x1_new, y1_new, x2_new, y2_new, pc_match, pc_match_flag)
            windows_annotation_list.append(value)
  
            obj = obj + 1    

        else:
            obj = obj + 1 
            
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
    windows_annotation_df = pd.DataFrame(windows_annotation_list, columns=column_name)
                
    return windows_annotation_df, intersection_flag

def windownsDefinition(img,
                       windows_width,
                       windows_height,
                       image_name,
                       annotation_df,
                       WINDOWS_NEGATIVE_PATH,
                       WINDOWS_POSITIVE_GERAL_PATH,
                       i_NoCow):
    
    largura_roi = windows_width
    altura_roi = windows_height
    
    altura, largura, canal = img.shape
    
    n_largura = int(largura/largura_roi)
    n_altura = int(altura/altura_roi)
    
    final_annotation_windows_list = []
    annotation_windows_list = []
    
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax', 'pc_match','pc_match_flag']
    final_annotation_windows_df = pd.DataFrame(final_annotation_windows_list, columns=column_name)
    
    i = 1
    x1 = 0
    y1 = 0
    x2 = largura_roi
    y2 = altura_roi
    
    for a in range(n_altura):
        for l in range(n_largura):
            
            #img_draft = cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 4)
            windows_img = img[y1:y2, x1:x2]
            
            windows_name = image_name + "_w_" + str(i)

            # FILTRO 1: Bounding box detection function
            # FILTRO 2: Bounding box overlapping area
            annotation_windows_df, intersection_flag = BoundingBoxDetectionFilter(image_name, windows_name, annotation_df, x1, y1, x2, y2, windows_width, windows_height)
                     
            if intersection_flag:

                final_annotation_windows_df = pd.concat([final_annotation_windows_df, annotation_windows_df])

                # EXPORT salva a imagem da windows COM intersecção no diretorio dos POSITIVOS: 
                exportWindows(windows_img, WINDOWS_POSITIVE_GERAL_PATH, windows_name)
                
            else:
                # EXPORT salva a imagem da windows SEM intersecção no diretorio dos NEGATIVOS: 
                exportWindowsNoCow(windows_img, WINDOWS_NEGATIVE_PATH, i_NoCow)
                i_NoCow = i_NoCow + 1




            i = i + 1
            x1 = x1 + largura_roi
            x2 = x2 + largura_roi
            l = l + 1           
        
        x1 = 0
        y1 = y2
        x2 = largura_roi
        y2 = y2 + altura_roi
        a = a + 1
    
    return final_annotation_windows_df, i_NoCow
                   

## Configurações e Constantes

In [ ]:
#Números divisores de 4000 entre 200 - 600.
WIN_WIDTH_LIST=[200, 250, 400, 500]

#Números divisores de 3000 entre 200 - 600.
WIN_HIGHT_LIST=[200, 250, 300, 375, 500, 600]

i=1

for win_hight in WIN_HIGHT_LIST:
    for win_width in WIN_WIDTH_LIST:
            i=i+1
            new_path = "C:\\NEW_TCC\\dataset\\janelamentos\\" + str(win_width) + "w_" + str(win_hight) +"h"
            createDir(new_path)

print("numero de combinações de janelas são:")            
print(i) 


In [ ]:
def main():
    
    # definição da dimensão das janelas
    WINDOWS_WIDTH = 200
    WINDOWS_HEIGHT = 200
    
    # declaração dos paths
    INPUT_DATASET_PATH = "C:\\NEW_TCC\\dataset\\selecionados"
    WINDOWS_IMAGE_ROOT_PATH = "C:\\NEW_TCC\\dataset\\windows_image"
    WINDOWS_NEGATIVE_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'negative')
    WINDOWS_POSITIVE_GERAL_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'geral')

    ORIGINAL_ANNOTATION_CSV_PATH = os.path.join(INPUT_DATASET_PATH, 'sel_annotation.csv')
    W_NEGATIVE_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_NEGATIVE_PATH, 'w_negative_annotation.csv')
    W_POSITIVE_GERAL_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_GERAL_PATH, 'w_positive_geral_annotation.csv')
    
    # Cria um Dataframe vazio para armazenar as annotations das janelas para todas as imagens
    final_annotation_windows_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag'] 
    final_annotation_windows_full_df = pd.DataFrame(final_annotation_windows_list, columns=column_name)
    
    # carrega o csv com as annotations originais
    annotation_df = pd.read_csv(ORIGINAL_ANNOTATION_CSV_PATH)
    
    # cria uma lista com o path de todas as imagens do diretorio de entrada
    image_names_list = createListNameImageFiles(INPUT_DATASET_PATH)
    
    # calcula o numero de imagens
    Qt_image = len(image_names_list)
    
    i_NoCow = 1
    
    # realiza o janelamento para cada imagem da lista
    for n in range(Qt_image):
       
        #Carrega imagem
        img = cv2.cvtColor(cv2.imread(image_names_list[n]), cv2.COLOR_BGR2RGB)
        
        # Extrai o nome da imagem a partir do diretorio
        image_name = createNameImage(image_names_list[n])
        
        # Realiza o janelamento da imagem e cria a respectiva anotação
        final_annotation_windows_df, i_NoCow = windownsDefinition(
            img,
            WINDOWS_WIDTH,
            WINDOWS_HEIGHT,
            image_name,
            annotation_df,
            WINDOWS_NEGATIVE_PATH,
            WINDOWS_POSITIVE_GERAL_PATH,
            i_NoCow)
        
        
        final_annotation_windows_full_df = pd.concat([final_annotation_windows_full_df, final_annotation_windows_df])
        
        
    # Salva o csv com as Annotations de todas as janelas
    final_annotation_windows_full_df = final_annotation_windows_full_df.reset_index(drop=True)
    final_annotation_windows_full_df.to_csv(W_POSITIVE_GERAL_ANNOTATION_CSV_PATH, encoding='utf-8', index=False)
    
    print("Processo executado com SUCESSO!")
    return image_name
main()

#### separa as janelas de acordo com a porcentagem de match com o bounding box

In [ ]:
def create_empty_df():
    # Cria um Dataframe vazio para armazenar as annotations das janelas para todas as imagens
    final_annotation_windows_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax', 'pc_match','pc_match_flag']
    final_annotation_windows_full_df = pd.DataFrame(final_annotation_windows_list, columns=column_name)
    
    return final_annotation_windows_full_df


def save_annotation_csv(df, OUTPUT_PATH):    
    # Salva o csv com as Annotations de todas as janelas
    df = df.reset_index(drop=True)
    df.to_csv(OUTPUT_PATH, encoding='utf-8', index=False)

# Separa as imagens do diretorio positivo geral de acordo com a porcentagem de match do boundbox e a windows
def main():
    INPUT_DATASET_PATH = "C:\\NEW_TCC\\dataset\\selecionados"
    WINDOWS_IMAGE_ROOT_PATH = "C:\\NEW_TCC\\dataset\\windows_image"
    WINDOWS_NEGATIVE_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'negative')
    WINDOWS_POSITIVE_MATCH_100_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_100')
    WINDOWS_POSITIVE_MATCH_95_MAIS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_95_mais')
    WINDOWS_POSITIVE_MATCH_80_MAIS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_80_mais')
    WINDOWS_POSITIVE_MATCH_80_MENOS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_80_menos')
    WINDOWS_POSITIVE_GERAL_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'geral')

    ORIGINAL_ANNOTATION_CSV_PATH = os.path.join(INPUT_DATASET_PATH, 'sel_annotation.csv')
    W_NEGATIVE_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_NEGATIVE_PATH, 'w_negative_annotation.csv')
    W_POSITIVE_GERAL_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_GERAL_PATH, 'w_positive_geral_annotation.csv')

    W_POSITIVE_100_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_100_PATH, 'w_positive_100_annotation.csv')
    W_POSITIVE_95_MAIS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_95_MAIS_PATH, 'w_positive_95_mais_annotation.csv')
    W_POSITIVE_80_MAIS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_80_MAIS_PATH, 'w_positive_80_mais_annotation.csv')
    W_POSITIVE_80_MENOS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_80_MENOS_PATH, 'w_positive_80_menos_annotation.csv')

    match_100_annotation_df = create_empty_df()
    match_95_mais_annotation_df = create_empty_df()
    match_80_mais_annotation_df = create_empty_df()
    match_80_menos_annotation_df = create_empty_df()
    
    
    # carrega o csv com as annotations originais
    annotation_df = pd.read_csv(W_POSITIVE_GERAL_ANNOTATION_CSV_PATH)
    
    match100df = annotation_df[annotation_df['pc_match_flag']=="MATCH_100"]
    match95maisdf = annotation_df[annotation_df['pc_match_flag']=="MATCH_95_MAIS"]
    match80maisdf = annotation_df[annotation_df['pc_match_flag']=="MATCH_80_MAIS"]
    
    # cria uma lista com o path de todas as imagens do diretorio de entrada
    image_names_list = createListNameImageFiles(WINDOWS_POSITIVE_GERAL_PATH)

    # calcula o numero de imagens
    Qt_image = len(image_names_list)
    
    match100List = list(match100df['filename'])
    match95maisList = list(match95maisdf['filename'])
    match80maisList = list(match80maisdf['filename'])


    import shutil
    # realiza o janelamento para cada imagem da lista
    for n in range(Qt_image):

        # Extrai o nome da imagem a partir do diretorio
        image_name = createNameImage(image_names_list[n])

        try:
            if image_name in match100List:

                old_path = image_names_list[n]
                new_path = os.path.join(WINDOWS_POSITIVE_MATCH_100_PATH,image_name+'.JPG')
                shutil.move(old_path, new_path)
                

                match_100_annotation_df = pd.concat([match_100_annotation_df, annotation_df[annotation_df['filename']==image_name]])
                



            if image_name in match95maisList:

                old_path = image_names_list[n]
                new_path = os.path.join(WINDOWS_POSITIVE_MATCH_95_MAIS_PATH,image_name+'.JPG')
                shutil.move(old_path, new_path)
                
                match_95_mais_annotation_df = pd.concat([match_95_mais_annotation_df, annotation_df[annotation_df['filename']==image_name]])
                


            if image_name in match80maisList:

                old_path = image_names_list[n]
                new_path = os.path.join(WINDOWS_POSITIVE_MATCH_80_MAIS_PATH,image_name+'.JPG')
                shutil.move(old_path, new_path)
                
                match_80_mais_annotation_df = pd.concat([match_80_mais_annotation_df, annotation_df[annotation_df['filename']==image_name]])
               

            old_path = image_names_list[n]
            new_path = os.path.join(WINDOWS_POSITIVE_MATCH_80_MENOS_PATH,image_name+'.JPG')
            shutil.move(old_path, new_path)
            
            match_80_menos_annotation_df = pd.concat([match_80_menos_annotation_df, annotation_df[annotation_df['filename']==image_name]])
               

        except:
            pass
        
    save_annotation_csv(match_100_annotation_df, W_POSITIVE_100_ANNOTATION_CSV_PATH)
    save_annotation_csv(match_95_mais_annotation_df, W_POSITIVE_95_MAIS_ANNOTATION_CSV_PATH)
    save_annotation_csv(match_80_mais_annotation_df, W_POSITIVE_80_MAIS_ANNOTATION_CSV_PATH)
    save_annotation_csv(match_80_menos_annotation_df, W_POSITIVE_80_MENOS_ANNOTATION_CSV_PATH)

    
    
main()    

#### Seleção da imagens usadas para o treinamento do modelo pós janelamento

In [ ]:
INPUT_DATASET_PATH = "C:\\NEW_TCC\\dataset\\selecionados"
WINDOWS_IMAGE_ROOT_PATH = "C:\\NEW_TCC\\dataset\\windows_image"
WINDOWS_NEGATIVE_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'negative')
WINDOWS_POSITIVE_MATCH_100_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_100')
WINDOWS_POSITIVE_MATCH_95_MAIS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_95_mais')
WINDOWS_POSITIVE_MATCH_80_MAIS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_80_mais')
WINDOWS_POSITIVE_MATCH_80_MENOS_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'match_80_menos')
WINDOWS_POSITIVE_GERAL_PATH = os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'positive', 'geral')

ORIGINAL_ANNOTATION_CSV_PATH = os.path.join(INPUT_DATASET_PATH, 'sel_annotation.csv')
W_NEGATIVE_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_NEGATIVE_PATH, 'w_negative_annotation.csv')
W_POSITIVE_GERAL_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_GERAL_PATH, 'w_positive_geral_annotation.csv')

W_POSITIVE_100_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_100_PATH, 'w_positive_100_annotation.csv')
W_POSITIVE_95_MAIS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_95_MAIS_PATH, 'w_positive_95_mais_annotation.csv')
W_POSITIVE_80_MAIS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_80_MAIS_PATH, 'w_positive_80_mais_annotation.csv')
W_POSITIVE_80_MENOS_ANNOTATION_CSV_PATH = os.path.join(WINDOWS_POSITIVE_MATCH_80_MENOS_PATH, 'w_positive_80_menos_annotation.csv')


In [ ]:
# ler os csvs com annotations
match_100_annotation_df = pd.read_csv(W_POSITIVE_100_ANNOTATION_CSV_PATH)
match_95_mais_annotation_df = pd.read_csv(W_POSITIVE_95_MAIS_ANNOTATION_CSV_PATH)
annotation_concated_df = pd.concat([match_100_annotation_df, match_95_mais_annotation_df])
annotation_concated_df

In [ ]:
# filtra registros onde pc_match>70%
print(len(annotation_concated_df))
annotation_filtered_df = annotation_concated_df[annotation_concated_df['pc_match']>70]
print(len(annotation_filtered_df))
annotation_filtered_df

In [ ]:
OUTPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\positive\\cow_dataset\\w_cow_annotations.csv"

# Salva o csv com as Annotations
df = annotation_filtered_df.reset_index(drop=True)
df.to_csv(OUTPUT_PATH, encoding='utf-8', index=False)

### Criação da classe NoCow

In [ ]:
WINDOWS_IMAGE_ROOT_PATH = "C:\\NEW_TCC\\dataset\\windows_image"
WINDOWS_NEGATIVE_PATH= os.path.join(WINDOWS_IMAGE_ROOT_PATH, 'negative')
COW_DATASET_PATH = os.path.join("C:\\NEW_TCC\\dataset\\windows_image\\positive\\cow_dataset")
NOCOW_DATASET_PATH = os.path.join(WINDOWS_NEGATIVE_PATH, 'nocow_dataset')
NOCOW_ANNOTATION_PATH = os.path.join(NOCOW_DATASET_PATH, 'w_nocow_annotations.csv')

In [ ]:
# cria uma lista com o path de todas as imagens do diretorio de entrada
cow_image_names_list = createListNameImageFiles(COW_DATASET_PATH)

# calcula o numero de imagens
Qt_image = len(cow_image_names_list)

# Cria uma lista com os paths de todas as imagens
nocow_image_names_list = createListNameImageFiles(WINDOWS_NEGATIVE_PATH)

nocow_dataset_image_list = random.sample(nocow_image_names_list, Qt_image)

print(len(nocow_image_names_list))
print(len(nocow_dataset_image_list))

for img in nocow_dataset_image_list:
    !copy {os.path.join(img)} {os.path.join(NOCOW_DATASET_PATH)} 
    
    
#criando as anotações do nocow


In [ ]:
WINDOWS_WIDTH = 500
WINDOWS_HEIGHT = 600

windows_width = WINDOWS_WIDTH
windows_height = WINDOWS_HEIGHT


# construir uma lista vazia
windows_annotation_list = []

# coletar nome das imagens
NameImageList = createListNameImageFiles(NOCOW_DATASET_PATH)

# loop para criar as anotações
for image_path in NameImageList:

    value = (createNameImage(image_path), windows_width, windows_height, "noCow", 0, 0, windows_width, windows_height, 0, 0)
    windows_annotation_list.append(value)
    
# preencher os df
column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax', 'pc_match','pc_match_flag']
noCowAnnotationDf = pd.DataFrame(windows_annotation_list, columns=column_name)

# salvar o df
noCowAnnotationDf = noCowAnnotationDf.reset_index(drop=True)
noCowAnnotationDf.to_csv(NOCOW_ANNOTATION_PATH, encoding='utf-8', index=False)

### Construção das anotações das imagens utilizadas no treinamento

In [ ]:
NOCOW_ANNOTATION_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\negative\\augmantation\\final_aug_annotation.csv"
COW_ANNOTATION_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\positive\\augmentation\\aug_1\\final_aug_annotation.csv"

FULL_ANNOTATION_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\annotations.csv"

In [ ]:
cow_annotation_df = pd.read_csv(COW_ANNOTATION_PATH)
nocow_annotation_df = pd.read_csv(NOCOW_ANNOTATION_PATH)
annotation_concated_df = pd.concat([cow_annotation_df, nocow_annotation_df])

# Salva o csv com as Annotations
df = annotation_concated_df.reset_index(drop=True)
df.to_csv(FULL_ANNOTATION_PATH, encoding='utf-8', index=False)

annotation_concated_df

## Separação em Treino e Teste

In [4]:
def create_empty_df():
    # Cria um Dataframe vazio para armazenar as annotations das janelas para todas as imagens
    final_annotation_windows_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax', 'pc_match','pc_match_flag']
    final_annotation_windows_full_df = pd.DataFrame(final_annotation_windows_list, columns=column_name)
    
    return final_annotation_windows_full_df

In [5]:
DIR_DATASET = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3"
DIRETORY_INPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\geral"
DIR_ANNOTATION_PATH = os.path.join(DIRETORY_INPUT_PATH, "annotations.csv")
DIR_TESTE_PATH = os.path.join(DIR_DATASET, "teste")
DIR_TREINO_PATH = os.path.join(DIR_DATASET, "treino")
FRACAO_TEST = 20

In [6]:
def main():
    import random
    import pandas as pd
    # carrega o csv com as annotations
    annotation_df = pd.read_csv(DIR_ANNOTATION_PATH)
    
    # Cria um Dataframe vazio para armazenar as annotations das Imagens de treino
    train_annotation_df = create_empty_df()
    
    # Cria um Dataframe vazio para armazenar as annotations das Imagens de teste
    test_annotation_df = create_empty_df()
    
    # Cria uma lista com os paths de todas as imagens
    image_names_list = createListNameImageFiles(DIRETORY_INPUT_PATH)
    
    Qt_image = len(image_names_list)
    
    n_test = int(Qt_image*FRACAO_TEST/100)
    
    test_list = random.sample(image_names_list, n_test)
    
    train_list = list(set(image_names_list) - set(test_list))
    
    for img in test_list:
        !copy {os.path.join(img)} {os.path.join(DIR_TESTE_PATH)}   
        
    # criar Test_Annotation_DF
    for path in test_list:
        image_name = createNameImage(path)
        imageAnnotationDf = annotation_df[annotation_df["filename"]==image_name]
        
        test_annotation_df = pd.concat([test_annotation_df, imageAnnotationDf])
        
    # Salva o csv com as Annotations das imagens de Teste
    test_annotation_df = test_annotation_df.reset_index(drop=True)
    test_annotation_df.to_csv(os.path.join(DIR_TESTE_PATH, "final_annotation_test.csv"), encoding='utf-8', index=False)
    
    for img in train_list:
        !copy {os.path.join(img)} {os.path.join(DIR_TREINO_PATH)} 
    
    # criar Train_Annotation_DF
    for path in train_list:
        image_name = createNameImage(path)
        imageAnnotationDf = annotation_df[annotation_df["filename"]==image_name]
        
        train_annotation_df = pd.concat([train_annotation_df, imageAnnotationDf])
        
    # Salva o csv com as Annotations das imagens de Treino
    train_annotation_df = train_annotation_df.reset_index(drop=True)
    train_annotation_df.to_csv(os.path.join(DIR_TREINO_PATH, "final_annotation_train.csv"), encoding='utf-8', index=False)
        
    

main()  

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

...

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 

        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).
        1 arquivo(s) copiado(s).


In [12]:
TEST_ANNOTATION_CSV_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\teste\\final_annotation_test.csv"
TRAIN_ANNOTATION_CSV_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\treino\\final_annotation_train.csv"

In [13]:
df_test = pd.read_csv(TEST_ANNOTATION_CSV_PATH)
df_train = pd.read_csv(TRAIN_ANNOTATION_CSV_PATH)

In [17]:
df_train['width']=500
df_train['height']=600

In [19]:
# Salva o csv com as Annotations das imagens de Treino
train_annotation_df = df_train.reset_index(drop=True)
train_annotation_df.to_csv(TRAIN_ANNOTATION_CSV_PATH, encoding='utf-8', index=False)

# Salva o csv com as Annotations das imagens de Treino
train_annotation_df = df_test.reset_index(drop=True)
train_annotation_df.to_csv(TEST_ANNOTATION_CSV_PATH, encoding='utf-8', index=False)

### Convertendo as annotation do formato Pascal para Yolo

In [20]:
def createYoloTxtFromPascalCsv(INPUT_PATH, OUTPUT_PATH, ANNOTATION_CSV_PATH):
    
    #Cria uma Lista com o Nome de todas as imagens no path de entrada 
    def createListNameImageFiles(input_path):

        from pathlib import Path
        images = Path(input_path).glob("*.jpg")
        image_names_list = [str(p) for p in images]

        return image_names_list

    # Esta função retonar apenas o nome da imagem a partir de um path
    def createNameImage(paths_image):
        name_format = paths_image.split("\\")[-1]
        image_name=name_format.split(".")[0] 

        return image_name
    
    def convertPascalToYolo(annotation_df, image_name):
        
        converted_df = annotation_df
        converted_df["x_center"] = ((annotation_df["xmax"]+annotation_df["xmin"])/2)/annotation_df["width"]
        converted_df["y_center"] = ((annotation_df["ymax"]+annotation_df["ymin"])/2)/annotation_df["height"]
        converted_df["width_yolo"] = (annotation_df["xmax"]-annotation_df["xmin"])/annotation_df["width"]
        converted_df["height_yolo"] = (annotation_df["ymax"]-annotation_df["ymin"])/annotation_df["height"]
        
        if image_name[0: 1]== "c":
            converted_df["class"] = 0
            
        else:
            converted_df["class"] = 1
        
        return converted_df
    
    
    annotation_df = pd.read_csv(ANNOTATION_CSV_PATH)
    paths_image_list = createListNameImageFiles(INPUT_PATH)
    for image_path in paths_image_list:
        image_name = createNameImage(image_path)
        converted_df = convertPascalToYolo(annotation_df, image_name)
        df = converted_df[converted_df["filename"] == image_name]
        df2 = df[["class", "x_center", "y_center", "width_yolo","height_yolo"]]
        txt_name = image_name + ".txt"
        np.savetxt(r''+OUTPUT_PATH+"\\"+txt_name, df2.values, fmt='%d %f %f %f %f')

    
    print("txts Salvos com sucesso!")

In [21]:
# teste
TEST_INPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\teste"
TEST_OUTPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\final\\test"
TEST_ANNOTATION_CSV_PATH = os.path.join(TEST_INPUT_PATH, "final_annotation_test.csv")

# treino
TREINO_INPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\treino"
TREINO_OUTPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\final\\train"
TREINO_ANNOTATION_CSV_PATH  = os.path.join(TREINO_INPUT_PATH, "final_annotation_train.csv")


createYoloTxtFromPascalCsv(TEST_INPUT_PATH, TEST_OUTPUT_PATH, TEST_ANNOTATION_CSV_PATH)
createYoloTxtFromPascalCsv(TREINO_INPUT_PATH, TREINO_OUTPUT_PATH, TREINO_ANNOTATION_CSV_PATH)

txts Salvos com sucesso!
txts Salvos com sucesso!


In [9]:
image_name = "cow567_w_36_aug_6"

In [10]:
ANNOTATION_CSV_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\full\\dataset_v3\\teste\\final_annotation_test.csv"

In [11]:
annotation_df = pd.read_csv(ANNOTATION_CSV_PATH)
df = annotation_df[annotation_df["filename"] == image_name]
df

,filename,width,height,class,xmin,ymin,xmax,ymax,pc_match,pc_match_flag
3960,cow567_w_36_aug_6,600,500,cow,443,556,500,600,0,0
